# Deep Neural Network for MNIST Classification


The dataset is called MNIST and refers to handwritten digit recognition. You can find more about it on Yann LeCun's website (Director of AI Research, Facebook). 

The dataset provides 70,000 images (28x28 pixels) of handwritten digits (1 digit per image). 

Since there are only 10 digits (0, 1, 2, 3, 4, 5, 6, 7, 8, 9), this is a classification problem with 10 classes. 

## Import the relevant packages

In [1]:
import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds

## Data

In [21]:
mnist_dataset, mnist_info = tfds.load(name="mnist", as_supervised=True, with_info=True)

In [47]:
mnist_train, mnist_test = mnist_dataset["train"], mnist_dataset["test"]

num_validation_samples = 0.1*mnist_info.splits["train"].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

num_test_samples = mnist_info.splits["test"].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.
    return image, label

scaled_train_and_validation_data = mnist_train.map(scale)
scaled_test_data = mnist_test.map(scale)

BUFFER_SIZE = 10000

shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

validation_data = shuffled_train_and_validation_data.take(num_validation_samples)
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)


BATCH_SIZE = 200

train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_validation_samples)
test_data = scaled_test_data.batch(num_test_samples)


validation_inputs, validation_targets = next(iter(validation_data))

## Model

### Outline the model

In [48]:
input_size = 784
output_size = 10
hidden_layer_size = 200

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28,1)),
    tf.keras.layers.Dense(hidden_layer_size, activation="relu"),
    tf.keras.layers.Dense(hidden_layer_size, activation="relu"),
    tf.keras.layers.Dense(output_size, activation="softmax")
])

### Optimizer and the loss function

In [49]:
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

### Training

In [50]:
NUM_EPOCHS = 5

model.fit(train_data, epochs = NUM_EPOCHS, validation_data = (validation_inputs, validation_targets), verbose=2)

Epoch 1/5
270/270 - 3s - loss: 0.3438 - accuracy: 0.9005 - val_loss: 0.1457 - val_accuracy: 0.9578
Epoch 2/5
270/270 - 4s - loss: 0.1315 - accuracy: 0.9610 - val_loss: 0.1051 - val_accuracy: 0.9683
Epoch 3/5
270/270 - 7s - loss: 0.0903 - accuracy: 0.9728 - val_loss: 0.0767 - val_accuracy: 0.9778
Epoch 4/5
270/270 - 9s - loss: 0.0631 - accuracy: 0.9808 - val_loss: 0.0654 - val_accuracy: 0.9820
Epoch 5/5
270/270 - 16s - loss: 0.0490 - accuracy: 0.9847 - val_loss: 0.0563 - val_accuracy: 0.9812


## Test the model

In [51]:
test_loss, test_accuracy = model.evaluate(test_data)
print('Test loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

1/1 [==============================] - 1s 578ms/step - loss: 0.0733 - accuracy: 0.9768
Test loss: 0.07. Test accuracy: 97.68%
